## Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

In [1]:
!curl localhost:9200

{
  "name" : "6a101724710a",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "jA-2MOTNSNa-iZWXl28Iuw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


What's the version.build_hash value?

`build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"`

## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [2]:
import requests

def get_documents(docs_url):
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    documents = []

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)
    return documents

In [3]:
from pprint import PrettyPrinter

pretty_print = PrettyPrinter(indent=2)

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
documents = get_documents(docs_url)

print(len(documents))
pretty_print.pprint(documents[0])

948
{ 'course': 'data-engineering-zoomcamp',
  'question': 'Course - When will the course start?',
  'section': 'General course-related questions',
  'text': 'The purpose of this document is to capture frequently asked '
          'technical questions\n'
          'The exact day and hour of the course will be 15th Jan 2024 at '
          "17h00. The course will start with the first  “Office Hours'' "
          'live.1\n'
          'Subscribe to course public Google Calendar (it works from Desktop '
          'only).\n'
          'Register before the course starts using this link.\n'
          'Join the course Telegram channel with announcements.\n'
          "Don’t forget to register in DataTalks.Club's Slack and join the "
          'channel.'}


In [4]:
pretty_print.pprint(documents[-1])

{ 'course': 'mlops-zoomcamp',
  'question': 'How to destroy infrastructure created via GitHub Actions',
  'section': 'Module 6: Best practices',
  'text': 'Problem description\n'
          'Infrastructure created in AWS with CD-Deploy Action needs to be '
          'destroyed\n'
          'Solution description\n'
          'From local:\n'
          'terraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" '
          '--reconfigure\n'
          'terraform destroy --var-file vars/prod.tfvars\n'
          'Added by Erick Calderin'}


## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client = Elasticsearch('http://localhost:9200') 

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-course-questions'})

In [8]:
def delete_es_index(index_name):
    # Delete the index
    try:
        response = es_client.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted successfully: {response}")
    except Exception as e:
        print(f"Error deleting index '{index_name}': {e}")


# delete_es_index(index_name)

In [9]:
from tqdm.auto import tqdm

/Users/patrickcmd/Projects/personal/llm-zoomcamp/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:06<00:00, 157.36it/s]


## Q3. Searching

We will execute a query "How do I execute a command in a running docker container?".

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

Look at the `_score` field.

In [11]:
query = 'How do I execute a command in a running docker container?'

In [12]:
def elastic_search_response(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    return response
    
    

In [13]:
es_response = elastic_search_response(query)

es_response

ObjectApiResponse({'took': 173, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 721, 'relation': 'eq'}, 'max_score': 81.70054, 'hits': [{'_index': 'homework-course-questions', '_id': 'cCC8fJABgim9gqxKeJ-0', '_score': 81.70054, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'homework-course-questions', '_id': 'fiC8fJABgim9gqxKbZ1o', '_score': 74.6618, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it runn

In [14]:
def get_top_ranking_score(response):
    return response["hits"].get("max_score")

top_ranking_score = get_top_ranking_score(es_response)

print(f"Top Ranking Score: {top_ranking_score}")

Top Ranking Score: 81.70054


In [34]:
es_response['hits']['hits'][0]['_score']

81.70054

In [37]:
def get_ranking_scores(response):
    ranking_scores = []
    for hit in response['hits']['hits']:
        ranking_scores.append(hit.get("_score"))
    return sorted(ranking_scores, reverse=True)

In [38]:
ranking_scores = get_ranking_scores(es_response)
print(f"Ranking scores: {ranking_scores}")

print(f"Top Ranking Score: {ranking_scores[0]}")

Ranking scores: [81.70054, 74.6618, 49.9899, 48.500305, 45.095573]
Top Ranking Score: 81.70054


## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?

In [15]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [16]:
from pprint import PrettyPrinter

pretty_print = PrettyPrinter(indent=2)

search_result = elastic_search(query)
pretty_print.pprint(search_result)

[ { 'course': 'machine-learning-zoomcamp',
    'question': 'How do I debug a docker container?',
    'section': '5. Deploying Machine Learning Models',
    'text': 'Launch the container image in interactive mode and overriding the '
            'entrypoint, so that it starts a bash command.\n'
            'docker run -it --entrypoint bash <image>\n'
            'If the container is already running, execute a command in the '
            'specific container:\n'
            'docker ps (find the container-id)\n'
            'docker exec -it <container-id> bash\n'
            '(Marcos MJD)'},
  { 'course': 'machine-learning-zoomcamp',
    'question': 'How do I copy files from my local machine to docker '
                'container?',
    'section': '5. Deploying Machine Learning Models',
    'text': 'You can copy files from your local machine into a Docker '
            "container using the docker cp command. Here's how to do it:\n"
            'To copy a file or directory from your local 

In [17]:
pretty_print.pprint(search_result[2])

{ 'course': 'machine-learning-zoomcamp',
  'question': 'How do I copy files from a different folder into docker '
              'container’s working directory?',
  'section': '5. Deploying Machine Learning Models',
  'text': 'You can copy files from your local machine into a Docker container '
          "using the docker cp command. Here's how to do it:\n"
          'In the Dockerfile, you can provide the folder containing the files '
          'that you want to copy over. The basic syntax is as follows:\n'
          'COPY ["src/predict.py", "models/xgb_model.bin", '
          '"./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan'}


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [18]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"{context_template.format(question=doc['question'], text=doc['text'])}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)

In [20]:
print(f"Prompt length: {len(prompt)}")

Prompt length: 1463


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.

In [22]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [27]:
query_tokens = encoding.encode(query)
print(query_tokens)

[5299, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 30]


In [28]:
print(len(query_tokens))

12


### Generating the answer 

In [29]:
from groq import Groq
import os

from dotenv import load_dotenv

load_dotenv()

True

In [30]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [31]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    
    return response.choices[0].message.content

In [32]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
print(rag(query))

Based on the CONTEXT, you can execute a command in a running Docker container using the `docker exec` command as follows:

docker exec -it <container-id> bash

Where `<container-id>` is the ID of the specific container you want to execute the command in. You can find the container ID by running `docker ps`.
